In [1]:
from neo4j import GraphDatabase
import csv
import pandas as pd, unicodedata, regex, json
from pathlib import Path
from ftfy import fix_text
from pathlib import Path
import pysbd
import regex as re 

# Ingesta

In [7]:
# Configura tus credenciales y URL de conexión
URI = "bolt://localhost:7687" 
USER = "neo4j"
PASSWORD = "password"

# Crear driver
driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

In [6]:
node_properties_query = """
MATCH (n)
UNWIND keys(n) AS prop
RETURN DISTINCT prop AS property_name
"""

with driver.session() as session:
    result = session.run(node_properties_query)
    columns = [record["property_name"] for record in result]

print(columns)


['doi', 'author_count', 'publication_date', 'abstract', 'title', 'scopus_id', 'neo4jImportId', 'affiliation_count', 'pk', 'name', 'country', 'city', 'auth_name', 'citation_count', 'initials', 'current_affiliation', 'first_name', 'last_name', 'updated', 'cursor', 'next_url']


In [8]:

def export_articles_to_csv():
    query = """
    MATCH (a:Article)
    WHERE a.scopus_id IS NOT NULL
      AND a.title IS NOT NULL AND a.title <> ""
      AND a.abstract IS NOT NULL AND a.abstract <> ""
      AND a.doi IS NOT NULL AND a.doi <> ""
    MATCH (au:Author)-[:WROTE]->(a)
    WITH a, collect(DISTINCT au.first_name + " " + au.last_name) AS authors
    WHERE size(authors) > 0
    OPTIONAL MATCH (a)-[:BELONGS_TO]->(af:Affiliation)
    WITH a, authors,
         collect(DISTINCT af.name)    AS affiliations,
         collect(DISTINCT af.city)    AS affiliation_cities,
         collect(DISTINCT af.country) AS affiliation_countries
    RETURN
      a.scopus_id                    AS scopus_id,
      a.title                        AS title,
      a.abstract                     AS abstract,
      a.doi                          AS doi,
      authors                        AS authors,
      affiliations                   AS affiliations,
      affiliation_cities             AS affiliation_cities,
      affiliation_countries          AS affiliation_countries,
      coalesce(a.citation_count, 0)  AS citation_count
    ORDER BY scopus_id
    """

    # Ejecuta consulta y arma DataFrame
    with driver.session() as session:
        rows = [dict(r) for r in session.run(query)]
    df = pd.DataFrame(rows)

    # Une listas con '; ' (evita introducir comas que confundan a quien lo lea a mano)
    def join_list(x):
        return "; ".join(str(v) for v in x if v) if isinstance(x, list) else x

    for col in ["authors", "affiliations", "affiliation_cities", "affiliation_countries"]:
        if col in df.columns:
            df[col] = df[col].map(join_list)

    # Orden de columnas
    df = df[[
        "title", "abstract", "doi", "authors",
        "affiliations", "affiliation_cities", "affiliation_countries",
        "citation_count", "scopus_id"
    ]]

    # Exporta con separador '|'
    # - quoting=QUOTE_MINIMAL: si algún campo contiene el separador '|', Pandas lo pondrá entre comillas.
    # - lineterminator="\n": EOL consistente.
    df.to_csv(
        "scopusdata.csv",
        index=False,
        encoding="utf-8",
        sep="|",
        quoting=csv.QUOTE_MINIMAL,
        lineterminator="\n",
    )
    print("Exportación completada: scopusdata.csv")

if __name__ == "__main__":
    export_articles_to_csv()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n    MATCH (a:Article)\n    WHERE a.scopus_id IS NOT NULL\n      AND a.title IS NOT NULL AND a.title <> ""\n      AND a.abstract IS NOT NULL AND a.abstract <> ""\n      AND a.doi IS NOT NULL AND a.doi <> ""\n    MATCH (au:Author)-[:WROTE]->(a)\n    WITH a, collect(DISTINCT au.first_name + " " + au.last_name) AS authors\n    WHERE size(authors) > 0\n    OPTIONAL MATCH (a)-[:BELONGS_TO]->(af:Affiliation)\n    WITH a, authors,\n         collect(DISTINCT af.name)    AS affiliations,\n         collect(DISTINCT af.city)    AS affiliation_cities,\n         collect(DISTINCT af.country) AS affiliation_countries\n    RETURN\n      a.scopus_id                    AS scopus_id,\n      a.title   

Exportación completada: scopusdata.csv


pseudocodigo

# normalizacion

In [ ]:

# -------- Configuración --------
INPUT_CSV   = "scopusdata.csv"      # archivo con separador '|'
OUTPUT_PATH = "processed.parquet"   # salida recomendada (parquet)
REMOVE_ISOLATED_NUMBERS = False     # True si quieres quitar números sueltos

# -------- Funciones --------
def normalize_unicode_and_case(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = fix_text(s)                        # corrige codificación/caracteres raros
    s = s.replace("\u00A0", " ")           # NBSP -> espacio normal
    s = unicodedata.normalize("NFC", s)    # Unicode canónica
    s = s.lower()                          # minúsculas
    return s

def strip_non_informative(s: str, remove_numbers: bool = False) -> str:
    # Conserva letras/números/espacios y signos básicos de textos científicos
    s = regex.sub(r"[^\p{L}\p{N}\s\-\.,;:()\[\]/%]", " ", s)
    if remove_numbers:
        # Elimina números aislados; conserva casos como "co2", "iso-9001"
        s = regex.sub(r"\b\d+\b", " ", s)
    s = regex.sub(r"\s+", " ", s).strip()  # espacios
    return s

def normalize_text(s: str, remove_numbers: bool = False) -> str:
    s = normalize_unicode_and_case(s)
    s = strip_non_informative(s, remove_numbers=remove_numbers)
    return s

def safe_convert_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """Convierte a dtypes 'seguros' sin depender de opciones globales."""
    try:
        return df.convert_dtypes(dtype_backend="numpy_nullable")  # pandas nuevos
    except TypeError:
        return df.convert_dtypes()  # pandas más viejos

def sanitize_objects(df: pd.DataFrame) -> pd.DataFrame:
    """Convierte objetos no escalares (listas/dicts) a JSON string."""
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].map(
                lambda x: x if isinstance(x, (str, int, float, bool, type(None)))
                else json.dumps(x, ensure_ascii=False)
            )
    return df

def try_save_parquet(df: pd.DataFrame, path: str) -> bool:
    """Intenta guardar con fastparquet, luego pyarrow. Devuelve True si logra parquet."""
    # 1) fastparquet
    try:
        import fastparquet  # noqa: F401
        df.to_parquet(path, index=False, engine="fastparquet", compression="gzip")
        return True
    except Exception:
        pass
    # 2) pyarrow
    try:
        import pyarrow  # noqa: F401
        df.to_parquet(path, index=False, engine="pyarrow")  # compresión por defecto
        return True
    except Exception:
        return False

# -------- Proceso --------
# Lee CSV con separador pipe. Si tu exportación puso comillas cuando había '|',
# pandas las respeta automáticamente.
df = pd.read_csv(INPUT_CSV, sep="|", encoding="utf-8")

# Asegura presencia de columnas requeridas
for col in ["title", "abstract"]:
    if col not in df.columns:
        df[col] = ""

# Normalización SOLO sobre texto analizable
df["title_norm"]    = df["title"].fillna("").map(lambda x: normalize_text(x, REMOVE_ISOLATED_NUMBERS))
df["abstract_norm"] = df["abstract"].fillna("").map(lambda x: normalize_text(x, REMOVE_ISOLATED_NUMBERS))

# Guardar salida con metadatos originales + columnas normalizadas
cols_out = list(df.columns)
for c in ["title_norm", "abstract_norm"]:
    if c not in cols_out:
        cols_out.append(c)

# Copia de salida + saneo de tipos
out = df[cols_out].copy()
out = safe_convert_dtypes(out)

# Asegura strings planos en columnas de texto clave
for c in ["title", "abstract", "title_norm", "abstract_norm"]:
    if c in out.columns:
        out[c] = out[c].astype(str)

# Serializa objetos complejos a JSON para evitar fallos de parquet
out = sanitize_objects(out)

# -------- Guardado robusto --------
parquet_ok = try_save_parquet(out, OUTPUT_PATH)

if parquet_ok:
    print(f"Listo → {Path(OUTPUT_PATH).resolve()}")
else:
    # Fallback a CSV para no perder progreso
    fallback = Path(OUTPUT_PATH).with_suffix(".csv")
    out.to_csv(fallback, index=False, sep="|", encoding="utf-8")
    print("No se pudo escribir Parquet con fastparquet ni pyarrow. "
          f"Se guardó CSV en → {fallback.resolve()}")

# Vista rápida
print(out[["title_norm", "abstract_norm"]].head(3))


Listo → /run/media/alech/backup/Github/tesis/processed.parquet
                                          title_norm  \
0  thou shalt not die in this place : an ethnomet...   
1  use of learning frames in climate change commu...   
2  free access to public ecuadorian universities:...   

                                       abstract_norm  
0  ecuador, located in south america, has a popul...  
1  differences in climate change learning frames ...  
2  a free higher education policy was implemented...  


### deteccion de idioma

In [ ]:
import pandas as pd
from pathlib import Path
from langdetect import detect, DetectorFactory

PATH = "processed.parquet"  
OUT  = "processed_lbl.parquet"   

DetectorFactory.seed = 0  # resultados más estables

def detect_lang_safe(text: str) -> str:
    t = (text or "").strip()
    if not t:
        return "und"  # indeterminado
    try:
        return detect(t)
    except Exception:
        return "und"

# --- cargar parquet ---
# intenta fastparquet y luego pyarrow
try:
    df = pd.read_parquet(PATH, engine="fastparquet")
except Exception:
    df = pd.read_parquet(PATH, engine="pyarrow")

# --- elegir fuente para detección
source_col = "abstract_norm" 
if source_col not in df.columns:
    # si no existe ninguna, crea vacía para no romper
    df[source_col] = ""

# --- detectar idioma ---
df["lang"] = df[source_col].map(detect_lang_safe)

# --- guardar ---
try:
    df.to_parquet(OUT, index=False, engine="fastparquet", compression="gzip")
except Exception:
    try:
        df.to_parquet(OUT, index=False, engine="pyarrow")
    except Exception:
        # último recurso: CSV para no perder el trabajo
        Path(OUT).with_suffix(".csv")
        df.to_csv(Path(OUT).with_suffix(".csv"), index=False, sep="|", encoding="utf-8")

print(df[["lang", source_col]].head(5))


  lang                                      abstract_norm
0   en  ecuador, located in south america, has a popul...
1   en  differences in climate change learning frames ...
2   en  a free higher education policy was implemented...
3   en  this study explored the influence of each fami...
4   en  the rapid adoption and the diversification of ...


## segmentar oraciones

In [4]:
# -------- Configuración --------
INPUT_PARQUET  = "processed_lbl.parquet"      # entrada
OUTPUT_PARQUET = "processed_sentences.parquet"  # salida
SOURCE_COL     = "abstract_norm"               # columna a segmentar

# -------- Carga robusta --------
def read_parquet_any(path: str) -> pd.DataFrame:
    try:
        return pd.read_parquet(path, engine="fastparquet")
    except Exception:
        return pd.read_parquet(path, engine="pyarrow")

# -------- Segmentadores (ES / EN) --------
seg_es = pysbd.Segmenter(language="es", clean=False)
seg_en = pysbd.Segmenter(language="en", clean=False)

def split_by_lang(text: str, lang: str = "es") -> list[str]:
    if not isinstance(text, str) or not text.strip():
        return []
    lang = (lang or "es").lower()
    seg = seg_en if lang.startswith("en") else seg_es
    try:
        return seg.segment(text.strip())
    except Exception:
        # fallback simple si falla pysbd
        return [text.strip()]

# -------- Proceso --------
df = read_parquet_any(INPUT_PARQUET)

if SOURCE_COL not in df.columns:
    df[SOURCE_COL] = ""

# si tienes columna de idioma, úsala; si no, asume "es"
lang_series = df["lang"] if "lang" in df.columns else ["es"] * len(df)

# segmentar
df["sentences"] = [
    split_by_lang(text, lang)
    for text, lang in zip(df[SOURCE_COL], lang_series)
]

# una oración por fila
out = df.explode("sentences", ignore_index=False)
out = out.rename(columns={"sentences": "sentence"})
out = out.reset_index(names="row_id_original")
out["sentence_idx"] = out.groupby("row_id_original").cumcount()

# columnas finales
keep = []
for c in ["scopus_id", "title", "abstract", "abstract_norm", "lang"]:
    if c in out.columns:
        keep.append(c)
keep += ["row_id_original", "sentence_idx", "sentence"]
out = out[keep]

# -------- Guardar --------
try:
    out.to_parquet(OUTPUT_PARQUET, index=False, engine="fastparquet", compression="gzip")
except Exception:
    try:
        out.to_parquet(OUTPUT_PARQUET, index=False, engine="pyarrow")
    except Exception:
        out.to_csv(Path(OUTPUT_PARQUET).with_suffix(".csv"), index=False, sep="|", encoding="utf-8")

print("Ejemplo:")
print(out[["row_id_original", "sentence_idx", "sentence"]].head(10))
print(f"\nGuardado → {Path(OUTPUT_PARQUET).resolve()}")

Ejemplo:
   row_id_original  sentence_idx  \
0                0             0   
1                0             1   
2                0             2   
3                0             3   
4                0             4   
5                0             5   
6                0             6   
7                0             7   
8                0             8   
9                0             9   

                                            sentence  
0  ecuador, located in south america, has a popul...  
1  according to the national institution of stati...  
2  palliative care and hospice are relatively new...  
3  in ecuador people usually die at home, in hosp...  
4  in 2012, the first ecuadorian hospice was crea...  
5  according to symbolic interactionism theory, r...  
6  symbolic interactionism proposes that human be...  
7  through an ethnomethodological approach, the f...  
8  results emerge from the introspection of real ...  
9  based on a thematic analysis, the followi

## tokenizacion

In [7]:
from gensim.models.phrases import Phrases, Phraser

INPUT_PARQUET  = "processed_sentences.parquet"
OUTPUT_PARQUET = "corpus_token"
SENT_COL = "sentence"

TOKEN_RE = re.compile(r"(?:[^\W_]+(?:[-_][^\W_]+)+|\d+(?:\.\d+)+|[^\W_]+)", re.VERBOSE | re.IGNORECASE | re.UNICODE)
def simple_tokenize(s:str):
    if not isinstance(s, str): return []
    return TOKEN_RE.findall(re.sub(r"\s+", " ", s.strip()))

# 1) Carga y tokeniza todas las oraciones
def read_parquet_any(p):
    try: return pd.read_parquet(p, engine="fastparquet")
    except Exception: return pd.read_parquet(p, engine="pyarrow")

df = read_parquet_any(INPUT_PARQUET)
df["tokens_base"] = df[SENT_COL].map(simple_tokenize)

# 2) Entrena bigramas y (opcional) trigramas
sentences = df["tokens_base"].tolist()

# ⚠️ delimiter must be str if tokens are str
bigram = Phrases(sentences, min_count=5, threshold=10.0, delimiter=" ")
bigram_phraser = Phraser(bigram)

trigram = Phrases(bigram_phraser[sentences], min_count=5, threshold=10.0, delimiter=" ")
trigram_phraser = Phraser(trigram)

# 3) Aplica: pega frases automáticamente (p.ej., aprendizaje_automático)
df["tokens"] = [trigram_phraser[bigram_phraser[toks]] for toks in df["tokens_base"]]
df["tokens_csv"] = df["tokens"].map(lambda xs: ",".join(xs))
df["n_tokens"] = df["tokens"].map(len)

# 4) Guarda
try:
    df.to_parquet(OUTPUT_PARQUET, index=False, engine="fastparquet", compression="gzip")
except Exception:
    try: df.to_parquet(OUTPUT_PARQUET, index=False, engine="pyarrow")
    except Exception: df.to_csv(Path(OUTPUT_PARQUET).with_suffix(".csv"), index=False, sep="|", encoding="utf-8")

print(df[[SENT_COL, "tokens_csv"]].head(8))

                                            sentence  \
0  ecuador, located in south america, has a popul...   
1  according to the national institution of stati...   
2  palliative care and hospice are relatively new...   
3  in ecuador people usually die at home, in hosp...   
4  in 2012, the first ecuadorian hospice was crea...   
5  according to symbolic interactionism theory, r...   
6  symbolic interactionism proposes that human be...   
7  through an ethnomethodological approach, the f...   

                                          tokens_csv  
0  ecuador,located,in south america,has,a,populat...  
1  according to the,national,institution,of,stati...  
2  palliative care,and,hospice,are,relatively new...  
3  in,ecuador,people,usually,die,at home,in,hospi...  
4   in,2012,the,first,ecuadorian,hospice,was created  
5  according to,symbolic,interactionism,theory,re...  
6  symbolic,interactionism,proposes,that,human be...  
7  through,an,ethnomethodological,approach,the,fo...  


### stop words

In [ ]:
# === Quitar stopwords sobre df["tokens"] con n-gramas separados por espacio ===
import re, unicodedata, os
import nltk
from nltk.corpus import stopwords as nltk_stop

# Asegura recurso stopwords NLTK
try:
    _ = nltk_stop.words("spanish")
except LookupError:
    nltk.download("stopwords")

# Idiomas a filtrar (ajusta a ["spanish"] si quieres solo ES)
LANGS = ["spanish", "english"]

# Construye set de stopwords
STOPSET = set()
for lang in LANGS:
    try:
        STOPSET |= set(nltk_stop.words(lang))
    except OSError:
        pass

def _norm(s: str) -> str:
    s = s.lower()
    s = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in s if not unicodedata.combining(ch))

STOPSET_NORM = {_norm(w) for w in STOPSET}

# Frases que no se filtran nunca (escribe aquí con ESPACIOS)
PROTECT_PHRASES = {
    "in south america",
    # añade más si quieres: "public health", "quality of life", ...
}

def is_stop(tok: str) -> bool:
    """
    Mantén n-gramas con contenido: elimina solo si TODAS las partes
    (separadas por espacio o guion) son stopwords; protege frases explícitas.
    """
    if not isinstance(tok, str) or not tok:
        return True  # vacío o no-string -> descartar

    t = _norm(tok).strip()

    # Protección explícita
    if t in PROTECT_PHRASES:
        return False

    # Token simple (sin espacios ni guiones)
    if (" " not in t) and ("-" not in t):
        return t in STOPSET_NORM

    # Token compuesto: separa por espacios o guiones (uno o más)
    parts = [p for p in re.split(r"[ \-]+", t) if p]
    if not parts:
        return True

    # Elimina SOLO si *todas* las partes son stopwords
    return all(p in STOPSET_NORM for p in parts)

def filter_tokens(tokens, min_len=2, drop_numeric=True):
    out = []
    if not isinstance(tokens, (list, tuple)):
        return out
    for t in tokens:
        if not isinstance(t, str) or not t:
            continue
        if drop_numeric and t.isnumeric():
            continue
        if len(t) < min_len:
            continue
        if is_stop(t):
            continue
        out.append(t)
    return out

# --- Aplicar al DF (requiere df["tokens"] como lista de strings) ---
if "tokens" not in df.columns:
    raise KeyError("Se esperaba df['tokens']. Asegúrate de haber generado los n-gramas antes.")

df["tokens_nostop"] = df["tokens"].map(filter_tokens)

# (Opcional) Texto para embeddings (bi-encoder): tokens unidos por espacio
df["text_for_embed"] = df["tokens_nostop"].map(lambda xs: " ".join(xs))

# Vistazo rápido (muestra si existen)
cols_show = [c for c in ["sentence", "tokens", "tokens_nostop", "text_for_embed"] if c in df.columns]
print(df[cols_show].head(8))

# --- Guardar a nuevo archivo para no sobrescribir el original ---
OUT_BASE = "corpus_token_nostop"
parquet_path = f"{OUT_BASE}.parquet"
try:
    df.to_parquet(parquet_path, index=False, engine="fastparquet", compression="gzip")
except Exception:
    try:
        df.to_parquet(parquet_path, index=False, engine="pyarrow")
    except Exception:
        df.to_csv(f"{OUT_BASE}.csv", index=False, sep="|", encoding="utf-8")

print("Guardado en:",
      parquet_path if os.path.exists(parquet_path) else f"{OUT_BASE}.csv")

                                            sentence  \
0  ecuador, located in south america, has a popul...   
1  according to the national institution of stati...   
2  palliative care and hospice are relatively new...   
3  in ecuador people usually die at home, in hosp...   
4  in 2012, the first ecuadorian hospice was crea...   
5  according to symbolic interactionism theory, r...   
6  symbolic interactionism proposes that human be...   
7  through an ethnomethodological approach, the f...   

                                              tokens  \
0  [ecuador, located, in south america, has, a, p...   
1  [according to the, national, institution, of, ...   
2  [palliative care, and, hospice, are, relativel...   
3  [in, ecuador, people, usually, die, at home, i...   
4  [in, 2012, the, first, ecuadorian, hospice, wa...   
5  [according to, symbolic, interactionism, theor...   
6  [symbolic, interactionism, proposes, that, hum...   
7  [through, an, ethnomethodological, approach,

## lemming

In [9]:
# LEMMATIZE text_for_embed (ES/EN) con spaCy en batch
import re, unicodedata, os
import pandas as pd
import spacy


df = pd.read_parquet("corpus_token_nostop.parquet")

assert "text_for_embed" in df.columns, "Falta la columna 'text_for_embed'."

# Carga modelos spaCy (puedes usar *_md/_lg si los tienes)
nlp_es = spacy.load("es_core_news_sm", disable=["parser","ner","textcat"])
nlp_en = spacy.load("en_core_web_sm", disable=["parser","ner","textcat"])

# Heurística ligera para detectar español
SPANISH_CUES = {"de","la","el","los","las","y","en","para","con","por","del","al","un","una","unos","unas","se","su","sus"}
ACCENTS_RE = re.compile(r"[áéíóúñüÁÉÍÓÚÑÜ]")

def _is_spanish_like(text: str) -> bool:
    if not isinstance(text, str):
        return False
    if ACCENTS_RE.search(text):
        return True
    words = [w.lower() for w in re.split(r"\s+", text.strip()) if w]
    # si al menos una palabra típica española aparece -> ES
    return any(w in SPANISH_CUES for w in words)

def _lemma_doc(doc):
    # cuida pronombres (algunos modelos antiguos devuelven "-PRON-")
    toks = []
    for t in doc:
        lem = t.lemma_ if t.lemma_ and t.lemma_ != "-PRON-" else t.text
        toks.append(lem.lower())
    # une con espacios (mismo formato que text_for_embed)
    return " ".join(toks)

# Divide índices por idioma con la heurística
idx_es = df.index[df["text_for_embed"].apply(_is_spanish_like)].tolist()
idx_en = df.index.difference(idx_es).tolist()

# Lematiza en batch por idioma (más rápido que fila a fila)
text_lemma = pd.Series(index=df.index, dtype=object)

if idx_es:
    docs_es = nlp_es.pipe(df.loc[idx_es, "text_for_embed"].fillna(""), batch_size=512, n_process=1)
    for i, doc in zip(idx_es, docs_es):
        text_lemma.loc[i] = _lemma_doc(doc)

if idx_en:
    docs_en = nlp_en.pipe(df.loc[idx_en, "text_for_embed"].fillna(""), batch_size=512, n_process=1)
    for i, doc in zip(idx_en, docs_en):
        text_lemma.loc[i] = _lemma_doc(doc)

# Asigna columna nueva
df["text_lemma"] = text_lemma.fillna("")

# Vista rápida
print(df[["text_for_embed", "text_lemma"]].head(8))

# Guardar (nuevo archivo para no pisar el anterior)
OUT = "corpus_token_nostop_lemma.parquet"
try:
    df.to_parquet(OUT, index=False, engine="fastparquet", compression="gzip")
except Exception:
    try:
        df.to_parquet(OUT, index=False, engine="pyarrow")
    except Exception:
        df.to_csv("corpus_token_nostop_lemma.csv", index=False, sep="|", encoding="utf-8")

print("Guardado en:", OUT if os.path.exists(OUT) else "corpus_token_nostop_lemma.csv")


                                      text_for_embed  \
0  ecuador located in south america population mi...   
1  according to the national institution statisti...   
2  palliative care hospice relatively new concept...   
3  ecuador people usually die at home hospitals n...   
4               first ecuadorian hospice was created   
5  according to symbolic interactionism theory re...   
6  symbolic interactionism proposes human beings ...   
7  ethnomethodological approach following researc...   

                                          text_lemma  
0  ecuador locate in south america population mil...  
1  accord to the national institution statistic e...  
2  palliative care hospice relatively new concept...  
3  ecuador people usually die at home hospital nu...  
4                 first ecuadorian hospice be create  
5  accord to symbolic interactionism theory resea...  
6  symbolic interactionism propose human being ca...  
7  ethnomethodological approach follow research a...  


## chunking 

In [4]:
# ====== CHUNKING *SIEMPRE* DESDE text_lemma (agrupado por scopus_id) ======
import os
import pandas as pd
import numpy as np
from transformers import AutoTokenizer

IN_PARQUET  = os.environ.get("IN_PARQUET", "corpus_token_nostop_lemma.parquet")
OUT_CHUNKS  = os.environ.get("OUT_CHUNKS", "corpus_chunks.parquet")

# --- Parámetros de chunking ---
MAX_TOKENS      = int(os.environ.get("MAX_TOKENS", "300"))      # 200–400 recomendado
OVERLAP_RATIO   = float(os.environ.get("OVERLAP_RATIO", "0.2"))  # 15–30% recomendado
OVERLAP_TOKENS  = int(MAX_TOKENS * OVERLAP_RATIO)
STRIDE          = max(1, MAX_TOKENS - OVERLAP_TOKENS)

# Tokenizer E5 (coherente con embeddings e5*)
TOKENIZER_NAME = "intfloat/multilingual-e5-base"
tok = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

# ---------- 0) Carga y validación ----------
df = pd.read_parquet(IN_PARQUET)

assert "scopus_id" in df.columns, "Falta columna 'scopus_id' en el parquet."
assert "text_lemma" in df.columns, "Falta columna 'text_lemma' (se usa siempre)."

# Normaliza tipos / limpieza básica
df["scopus_id"]  = df["scopus_id"].astype(str)
df["text_lemma"] = (
    df["text_lemma"].fillna("").astype(str)
      .str.replace(r"\s+", " ", regex=True).str.strip()
)

# ---------- 1) Construir texto lematizado por documento ----------
# Orden preferido: sentence_idx > row_id_original > orden actual
sort_keys = ["scopus_id"]
if "sentence_idx" in df.columns:
    sort_keys += ["sentence_idx"]
elif "row_id_original" in df.columns:
    sort_keys += ["row_id_original"]

df = df.sort_values(sort_keys, kind="mergesort")

# Texto lematizado consolidado por scopus_id
agg_text = df.groupby("scopus_id")["text_lemma"].apply(
    lambda s: " ".join([t for t in s.astype(str) if t])
).rename("text_for_chunk")

# Solo scopus_id + texto consolidado
doc_df = agg_text.to_frame().reset_index()
doc_df["doc_id"] = np.arange(len(doc_df), dtype="int64")

# Limpieza final del texto
doc_df["text_for_chunk"] = (
    doc_df["text_for_chunk"].fillna("").astype(str)
       .str.replace(r"\s+", " ", regex=True).str.strip()
)
doc_df = doc_df[doc_df["text_for_chunk"].str.len() > 0].reset_index(drop=True)

# ---------- 2) Chunker por tokens ----------
def chunk_text_by_tokens(text: str, max_tokens: int = MAX_TOKENS, stride: int = STRIDE):
    ids = tok.encode(text, add_special_tokens=False)
    n = len(ids)
    if n == 0:
        return []
    chunks = []
    start = 0
    while start < n:
        end = min(start + max_tokens, n)
        sl = ids[start:end]
        chunk_txt = tok.decode(sl, skip_special_tokens=True).strip()
        if chunk_txt:
            chunks.append({
                "start_token": start,
                "end_token": end,
                "token_count": end - start,
                "text_chunk": chunk_txt
            })
        if end == n:
            break
        start += stride
    return chunks

# ---------- 3) Generar filas de chunks (solo campos mínimos + scopus_id) ----------
rows = []
for _, r in doc_df.iterrows():
    doc_id = int(r["doc_id"])
    scid   = str(r["scopus_id"])
    text   = r["text_for_chunk"]
    for j, ch in enumerate(chunk_text_by_tokens(text, MAX_TOKENS, STRIDE)):
        rows.append({
            "doc_id": doc_id,
            "chunk_id": j,
            "chunk_uid": f"{doc_id}-{j}",
            "scopus_id": scid,
            "start_token": ch["start_token"],
            "end_token": ch["end_token"],
            "token_count": ch["token_count"],
            "text_chunk": ch["text_chunk"],
        })

chunks_df = pd.DataFrame(rows)

print(chunks_df.head(8))
print("N docs (únicos scopus_id):", doc_df.shape[0], "| N chunks:", chunks_df.shape[0])

# ---------- 4) Guardar (solo mínimos) ----------
save_cols = [
    "doc_id","chunk_id","chunk_uid","scopus_id",
    "start_token","end_token","token_count","text_chunk"
]
chunks_out = chunks_df[save_cols].copy()

# Parquet (pyarrow preferente)
try:
    chunks_out.to_parquet(OUT_CHUNKS, index=False, engine="pyarrow")
except Exception:
    try:
        chunks_out.to_parquet(OUT_CHUNKS, index=False, engine="fastparquet", compression="gzip")
    except Exception:
        chunks_out.to_csv(OUT_CHUNKS.replace(".parquet", ".csv"), index=False, sep="|", encoding="utf-8")

print("Guardado en:", OUT_CHUNKS)


Token indices sequence length is longer than the specified maximum sequence length for this model (538 > 512). Running this sequence through the model will result in indexing errors


   doc_id  chunk_id chunk_uid    scopus_id  start_token  end_token  \
0       0         0       0-0  85059061481            0        156   
1       1         0       1-0  85061967853            0         94   
2       2         0       2-0  85067792389            0         60   
3       3         0       3-0  85068192726            0        255   
4       4         0       4-0  85069901345            0        173   
5       5         0       5-0  85070472925            0        205   
6       6         0       6-0  85071977997            0        195   
7       7         0       7-0  85072017885            0        169   

   token_count                                         text_chunk  
0          156  ecuador locate in south america population mil...  
1           94  difference climate change learning frame pedag...  
2           60  free high education policy be implement ecuado...  
3          255  this study explore influence family member lif...  
4          173  rapid adoptio

In [10]:
import os, gc, pickle, numpy as np, pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from typing import Optional

# -------------------- Config --------------------
PARQUET_PATH   = os.environ.get("PARQUET_PATH", "corpus_chunks.parquet")   # <-- solo contiene los chunks
FAISS_PATH     = os.environ.get("FAISS_PATH", "faiss_index_ip.bin")
PKL_MIN_PATH   = os.environ.get("PKL_MIN_PATH", "embeddings_meta_min.pkl")


# Modelo recomendado para CPU
EMB_MODEL        = os.environ.get("EMB_MODEL", "intfloat/multilingual-e5-small")
EMB_MAX_SEQ_LEN  = int(os.environ.get("EMB_MAX_SEQ_LEN", "300"))   # <=512
INIT_BATCH       = int(os.environ.get("BATCH_SIZE", "32"))
MIN_BATCH        = 1

# -------------------- 0) Carga parquet de chunks + limpieza --------------------
try:
    chunks_df = pd.read_parquet(PARQUET_PATH, engine="pyarrow")
except Exception:
    chunks_df = pd.read_parquet(PARQUET_PATH, engine="fastparquet")

required_cols = {"doc_id","chunk_id","start_token","end_token","text_chunk"}
missing = required_cols - set(chunks_df.columns)
if missing:
    raise ValueError(f"Faltan columnas requeridas en {PARQUET_PATH}: {missing}")

# Normaliza tipos
for c in ["doc_id","chunk_id","start_token","end_token"]:
    if chunks_df[c].dtype.kind not in "iu":
        chunks_df[c] = pd.to_numeric(chunks_df[c], errors="coerce").fillna(0).astype("int64")

# Limpieza texto
chunks_df["text_chunk"] = chunks_df["text_chunk"].astype(str).str.strip()
chunks_df = chunks_df[chunks_df["text_chunk"].str.len() > 0].reset_index(drop=True)

# chunk_uid y scopus_id opcional
if "chunk_uid" not in chunks_df.columns:
    chunks_df["chunk_uid"] = chunks_df["doc_id"].astype(str) + "-" + chunks_df["chunk_id"].astype(str)
if "scopus_id" in chunks_df.columns:
    chunks_df["scopus_id"] = chunks_df["scopus_id"].astype(str)

# IDs vectoriales alineados 0..N-1
N = len(chunks_df)
chunks_df["vec_id"] = np.arange(N, dtype="int64")
chunks_df["embedding_model"] = EMB_MODEL

# -------------------- 1) Modelo (CPU) alineado con chunking --------------------
print(f"[INFO] device encode: cpu; model={EMB_MODEL}")
model = SentenceTransformer(EMB_MODEL, device="cpu")
model.max_seq_length = min(EMB_MAX_SEQ_LEN, 512)
print(f"[INFO] model.max_seq_length = {model.max_seq_length}")

# Prefijo E5
passages = ("passage: " + chunks_df["text_chunk"]).tolist()

# -------------------- 2) FAISS (IP con embeddings normalizados -> coseno) --------------------
def make_faiss_index(dim: int):
    print("[INFO] FAISS-CPU")
    return faiss.IndexFlatIP(dim)

def st_encode_cpu(texts, batch_size, normalize=True, to_numpy=True):
    embs = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=False,
        normalize_embeddings=normalize,
        convert_to_numpy=to_numpy
    )
    return np.asarray(embs, dtype="float32")

# -------------------- 3) Bucle por lotes (CPU) --------------------
def stream_encode_and_build(passages, init_bs=INIT_BATCH, min_bs=MIN_BATCH):
    i, bs = 0, init_bs
    index = None
    dim = None

    while i < N:
        j = min(i + bs, N)
        batch = passages[i:j]
        try:
            emb = st_encode_cpu(batch, batch_size=bs, normalize=True, to_numpy=True)

            if dim is None:
                dim = emb.shape[1]
                index = make_faiss_index(dim)

            index.add(emb)

            i = j
            print(f"[PROG] {i}/{N} ({100.0*i/N:.1f}%) - bs={bs}")

        except Exception as e:
            prev_bs = bs
            bs = max(min_bs, bs // 2)
            gc.collect()
            if prev_bs == bs and bs == min_bs:
                raise RuntimeError(f"Fallo persistente en CPU con batch={bs}: {e}") from e
            print(f"[WARN] Error en i={i}. Bajo batch {prev_bs}->{bs} y reintento…")
            continue

    return index, dim

# -------------------- 4) Ejecutar pipeline --------------------
index_cpu, dim = stream_encode_and_build(passages)

# -------------------- 5) Guardar FAISS + PKL (mapa mínimo) --------------------
faiss.write_index(index_cpu, FAISS_PATH)
print(f"[OK] FAISS guardado: {FAISS_PATH} | ntotal={index_cpu.ntotal} | dim={dim}")

# PKL: guardar meta_min con scopus_id si existe
min_cols = ["vec_id","chunk_uid","doc_id","chunk_id","start_token","end_token"]
if "scopus_id" in chunks_df.columns:
    min_cols.append("scopus_id")

with open(PKL_MIN_PATH, "wb") as f:
    pickle.dump({
        "model": EMB_MODEL,
        "device_used": "cpu",
        "dim": dim,
        "meta_min": chunks_df[min_cols].copy()
    }, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f"[OK] PKL (meta_min) guardado: {PKL_MIN_PATH}")


[INFO] device encode: cpu; model=intfloat/multilingual-e5-small
[INFO] model.max_seq_length = 300
[INFO] FAISS-CPU
[PROG] 32/21005 (0.2%) - bs=32
[PROG] 64/21005 (0.3%) - bs=32
[PROG] 96/21005 (0.5%) - bs=32
[PROG] 128/21005 (0.6%) - bs=32
[PROG] 160/21005 (0.8%) - bs=32
[PROG] 192/21005 (0.9%) - bs=32
[PROG] 224/21005 (1.1%) - bs=32
[PROG] 256/21005 (1.2%) - bs=32
[PROG] 288/21005 (1.4%) - bs=32
[PROG] 320/21005 (1.5%) - bs=32
[PROG] 352/21005 (1.7%) - bs=32
[PROG] 384/21005 (1.8%) - bs=32
[PROG] 416/21005 (2.0%) - bs=32
[PROG] 448/21005 (2.1%) - bs=32
[PROG] 480/21005 (2.3%) - bs=32
[PROG] 512/21005 (2.4%) - bs=32
[PROG] 544/21005 (2.6%) - bs=32
[PROG] 576/21005 (2.7%) - bs=32
[PROG] 608/21005 (2.9%) - bs=32
[PROG] 640/21005 (3.0%) - bs=32
[PROG] 672/21005 (3.2%) - bs=32
[PROG] 704/21005 (3.4%) - bs=32
[PROG] 736/21005 (3.5%) - bs=32
[PROG] 768/21005 (3.7%) - bs=32
[PROG] 800/21005 (3.8%) - bs=32
[PROG] 832/21005 (4.0%) - bs=32
[PROG] 864/21005 (4.1%) - bs=32
[PROG] 896/21005 (4.3%) 

## recuperacion

In [ ]:
import os, pickle
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

# ---- Rutas (ajústalas o usa variables de entorno) ----
PKL_MIN_PATH = os.environ.get("PKL_MIN_PATH", "embeddings_meta_min.pkl")
FAISS_PATH   = os.environ.get("FAISS_PATH", "faiss_index_ip.bin")
SCOPUS_CSV   = os.environ.get("SCOPUS_CSV", "scopusdata.csv")
SCOPUS_SEP   = os.environ.get("SCOPUS_SEP", "|") 

# ---- Caches simples ----
_model_cache = None
_meta_min_cache = None
_index_cache = None
_scopus_cache = None

def load_pkl_and_model(emb_max_seq_len=300):
    global _model_cache, _meta_min_cache
    if _model_cache is not None and _meta_min_cache is not None:
        return _model_cache, _meta_min_cache
    with open(PKL_MIN_PATH, "rb") as f:
        pkl = pickle.load(f)

    meta_min = pkl["meta_min"].copy()  # DataFrame: vec_id, chunk_uid, doc_id, chunk_id, (scopus_id), start/end
    _meta_min_cache = meta_min

    model_name = pkl.get("model", "intfloat/multilingual-e5-small")
    model = SentenceTransformer(model_name, device="cpu")
    model.max_seq_length = min(int(emb_max_seq_len), 512)
    _model_cache = model

    print(f"[INFO] Modelo: {model_name} | max_seq_length={model.max_seq_length}")
    print(f"[INFO] meta_min columnas: {list(meta_min.columns)} | filas={len(meta_min)}")
    return _model_cache, _meta_min_cache

def load_faiss():
    global _index_cache
    if _index_cache is None:
        _index_cache = faiss.read_index(FAISS_PATH)
        print(f"[INFO] Índice FAISS cargado: ntotal={_index_cache.ntotal}")
    return _index_cache

def load_scopus_csv():
    global _scopus_cache
    if _scopus_cache is None:
        df = pd.read_csv(SCOPUS_CSV, sep=SCOPUS_SEP)
        if "scopus_id" not in df.columns:
            raise ValueError(f"{SCOPUS_CSV} no tiene columna 'scopus_id'")
        df["scopus_id"] = df["scopus_id"].astype(str)
        _scopus_cache = df
        print(f"[INFO] scoupusdata.csv: filas={len(df)} | cols={len(df.columns)}")
    return _scopus_cache

def e5_encode_query(model, query_text: str):
    return model.encode([f"query: {query_text}"],
                        normalize_embeddings=True,
                        convert_to_numpy=True).astype("float32")

def search_min(query_text: str, topk: int = 5) -> pd.DataFrame:
    """
    Devuelve SOLO el meta mínimo del PKL (sin CSV):
    vec_id, score, chunk_uid, doc_id, chunk_id, (scopus_id si existe), start/end
    """
    model, meta_min = load_pkl_and_model()
    index = load_faiss()

    q = e5_encode_query(model, query_text)
    D, I = index.search(q, topk)
    vec_ids = I[0].tolist()

    hits = meta_min.set_index("vec_id").loc[vec_ids].reset_index()
    hits.insert(1, "score", D[0])

    cols_front = [c for c in ["vec_id","score","chunk_uid","doc_id","chunk_id","scopus_id","start_token","end_token"] if c in hits.columns]
    rest = [c for c in hits.columns if c not in cols_front]
    return hits[cols_front + rest].reset_index(drop=True)

def search_full_scopus(query_text: str, topk: int = 5) -> pd.DataFrame:
    """
    Une el TOP-K con TODAS las columnas de scoupusdata.csv por scopus_id.
    """
    model, meta_min = load_pkl_and_model()
    index = load_faiss()
    sc = load_scopus_csv()

    q = e5_encode_query(model, query_text)
    D, I = index.search(q, topk)
    vec_ids = I[0].tolist()

    hits = meta_min.set_index("vec_id").loc[vec_ids].reset_index()
    hits.insert(1, "score", D[0])

    if "scopus_id" not in hits.columns:
        raise ValueError("meta_min en PKL no contiene 'scopus_id'; no puedo unir con el CSV.")

    out = hits.merge(sc, how="left", on="scopus_id")

    # Orden: primero claves/score/offsets, luego TODO el CSV
    front = [c for c in ["vec_id","score","chunk_uid","doc_id","chunk_id","scopus_id","start_token","end_token"] if c in out.columns]
    csv_cols = [c for c in sc.columns if c not in front]
    return out[front + csv_cols].reset_index(drop=True)

# ====== DEMO RÁPIDA ======
if __name__ == "__main__":
    query = "Ecuador"   # cambia por tu consulta
    print("\n=== TOP-K (meta mínimo) ===")
    print(search_min(query, topk=5))

    print("\n=== TOP-K + TODA la metadata del CSV ===")
    df = search_full_scopus(query, topk=5)
    # Si quieres guardar para revisar en Excel:
    # df.to_csv("search_results_full_scopus.csv", index=False, encoding="utf-8")
    print(df.head(5))



=== TOP-K (meta mínimo) ===
[INFO] Modelo: intfloat/multilingual-e5-small | max_seq_length=300
[INFO] meta_min columnas: ['vec_id', 'chunk_uid', 'doc_id', 'chunk_id', 'start_token', 'end_token', 'scopus_id'] | filas=21005
[INFO] Índice FAISS cargado: ntotal=21005
   vec_id     score chunk_uid  doc_id  chunk_id    scopus_id  start_token  \
0    3933  0.874905    3534-0    3534         0  85126742544            0   
1   11456  0.867839   10461-0   10461         0  85161272258            0   
2    7683  0.866706    6982-0    6982         0  85142396040            0   
3   10501  0.866331    9586-0    9586         0  85152377788            0   
4   20104  0.866028   18406-0   18406         0  85198537488            0   

   end_token  
0        137  
1        103  
2         65  
3        158  
4        168  

=== TOP-K + TODA la metadata del CSV ===
[INFO] scoupusdata.csv: filas=19233 | cols=9
   vec_id     score chunk_uid  doc_id  chunk_id    scopus_id  start_token  \
0    3933  0.87490